In [ ]:
from sklearn.feature_selection import chi2,SelectFdr,SelectFpr,SelectKBest,SelectPercentile,SelectFwe,mutual_info_classif,GenericUnivariateSelect
from sklearn.datasets import load_iris,load_breast_cancer



In [12]:
from sklearn.datasets import load_iris,load_breast_cancer
x,y = load_breast_cancer(return_X_y=True)

print(x.shape)
# result = x.feature_names

# print(result)

(569, 30)


In [24]:
from sklearn.feature_selection import SelectKBest,chi2,SelectPercentile,SelectFdr,SelectFpr,SelectFwe
k_chi = SelectKBest(chi2,k=15).fit_transform(x,y)
p_chi = SelectPercentile(chi2,percentile=10).fit_transform(x,y)
fdr_chi = SelectFdr(chi2,alpha=0.05).fit_transform(x,y)
fpr_chi = SelectFpr(chi2,alpha=0.05).fit_transform(x,y)
fwe_chi = SelectFwe(chi2,alpha=0.05).fit_transform(x,y)


print("original data",x.shape)

print("kBest chi2",k_chi.shape)
print("percentile chi2",p_chi.shape)
print("fdr chi2",fdr_chi.shape)
print("fpr chi2",fpr_chi.shape)
print("fwe chi2",fwe_chi.shape)







original data (569, 30)
kBest chi2 (569, 15)
percentile chi2 (569, 3)
fdr chi2 (569, 17)
fpr chi2 (569, 17)
fwe chi2 (569, 16)


In [35]:
from sklearn.feature_selection import SelectKBest,chi2,SelectPercentile,SelectFdr,SelectFpr,SelectFwe,mutual_info_classif,f_classif

k_mutual = SelectKBest(mutual_info_classif,k=15).fit_transform(x,y)
p_mutual = SelectPercentile(mutual_info_classif,percentile=10).fit_transform(x,y)
# Selector = SelectFdr(mutual_info_classif,alpha=0.05).fit(x,y)
# fdr_mutual = Selector.transform(x)
# fpr_mutual = SelectFpr(mutual_info_classif,alpha=0.05).fit_transform(x,y)
# fwe_mutual = SelectFwe(mutual_info_classif,alpha=0.05).fit_transform(x,y)


print("original data",x.shape)
# print(k_mutual.get_support())
print("kBest mutual",k_mutual.shape)
print("percentile mutual",p_mutual.shape)
# print("fdr mutual",fdr_mutual.shape)
# print("fpr mutual",fpr_mutual.shape)
# print("fwe mutual",fwe_mutual.shape)




original data (569, 30)
kBest mutual (569, 15)
percentile mutual (569, 3)


### Gpt mutual info classif

The error occurs because SelectFdr, SelectFpr, and SelectFwe require p-values for feature selection, but mutual_info_classif only returns scores, not p-values.

Why It Doesn't Work Directly
mutual_info_classif → Outputs dependency scores (higher = better).

SelectFdr/Fpr/Fwe → Require p-values (lower = better).

In [36]:
import numpy as np
from sklearn.feature_selection import mutual_info_classif, SelectFdr, SelectFpr, SelectFwe

# Compute mutual info scores
mi_scores = mutual_info_classif(x, y)

# Simulate p-values (since mutual_info_classif doesn't provide them)
normalized_scores = mi_scores / np.max(mi_scores)  # Scale to [0,1]
p_values = 1 - normalized_scores  # Lower score → Higher p-value

# Define a dummy function that returns (p_values, dummy_scores)
def mock_pval_func(x, y):
    return (p_values, np.zeros_like(p_values))

# Now apply FDR/FPR/FWE selection
selector_fdr = SelectFdr(mock_pval_func, alpha=0.05).fit(x, y)
selector_fpr = SelectFpr(mock_pval_func, alpha=0.05).fit(x, y)
selector_fwe = SelectFwe(mock_pval_func, alpha=0.05).fit(x, y)

# Get selected features
X_fdr = selector_fdr.transform(x)
X_fpr = selector_fpr.transform(x)
X_fwe = selector_fwe.transform(x)

print("FDR selected:", X_fdr.shape)
print("FPR selected:", X_fpr.shape)
print("FWE selected:", X_fwe.shape)

FDR selected: (569, 30)
FPR selected: (569, 30)
FWE selected: (569, 30)


### Generic univarite selection

selected is the transformed NumPy array (output of .transform()), which does not have .get_support().

Instead, you should call .get_support() on the fitted selector object (generic_univariate).

In [51]:
from sklearn.feature_selection import GenericUnivariateSelect
generic_univariate = GenericUnivariateSelect(mutual_info_classif,mode="percentile",param=50).fit(x,y)
selected = generic_univariate.transform(x)
# print(generic_univariate.shape)
inputs = load_breast_cancer()
feature =inputs.feature_names
print(feature[generic_univariate.get_support()])
print(x.shape)


['mean radius' 'mean perimeter' 'mean area' 'mean compactness'
 'mean concavity' 'mean concave points' 'radius error' 'perimeter error'
 'area error' 'worst radius' 'worst perimeter' 'worst area'
 'worst compactness' 'worst concavity' 'worst concave points']
(569, 30)


AttributeError: 'numpy.ndarray' object has no attribute 'feature_names'